<a href="https://colab.research.google.com/github/Jatinnagar777/demo/blob/main/Movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import difflib
from sklearn.metrics.pairwise import cosine_similarity

#Datasets

In [ ]:
df = pd.read_csv('movies.csv')
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
#number of rows and columns in datasets
df.shape

(4803, 24)

#Data preprocessing

In [ ]:
#features selection
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director', 'overview']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director', 'overview']


In [ ]:
#Replacing the null values with null string
for col in selected_features:
  df[col] = df[col].fillna('')

In [ ]:
#Combining all the selected features
combined_features = df['genres']+' '+df['keywords']+' '+df['tagline']+' '+df['cast']+' '+df['director']+' '+df['overview']

In [ ]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [ ]:
#Converting text data to features vectors
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)

In [ ]:
print(feature_vectors)

  (0, 5245)	0.16506865163441017
  (0, 1014)	0.12285843797047788
  (0, 1243)	0.05270709034262742
  (0, 21523)	0.16650106431669984
  (0, 1281)	0.0319519027005067
  (0, 19650)	0.16120557402371172
  (0, 10368)	0.138482776543016
  (0, 2974)	0.09974129089792808
  (0, 27515)	0.1456922041909068
  (0, 2685)	0.09805341831299118
  (0, 4129)	0.06195738171764886
  (0, 18025)	0.1093940665047336
  (0, 28597)	0.15490753406550084
  (0, 19541)	0.04908322232312577
  (0, 18249)	0.14769810641761005
  (0, 27405)	0.031015019219557106
  (0, 7827)	0.17979882024342952
  (0, 14023)	0.04170092858483918
  (0, 17021)	0.14324534514274756
  (0, 20104)	0.19260031442011957
  (0, 4768)	0.12501550204808318
  (0, 239)	0.19716936546022965
  (0, 13474)	0.036478409857958735
  (0, 4288)	0.13213124585063998
  (0, 14180)	0.08418056281586365
  :	:
  (4802, 27153)	0.05940079357098276
  (4802, 19243)	0.061349419053434766
  (4802, 11941)	0.07829183421073846
  (4802, 29680)	0.07735547381237222
  (4802, 9346)	0.07104321122746965
  (4

Cosine Similarity

In [ ]:
#getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors)

In [ ]:
print(similarity)

[[1.         0.05083168 0.0332947  ... 0.02749812 0.0304889  0.0072518 ]
 [0.05083168 1.         0.04356836 ... 0.05077045 0.03100979 0.01521198]
 [0.0332947  0.04356836 1.         ... 0.02646984 0.04751623 0.01372603]
 ...
 [0.02749812 0.05077045 0.02646984 ... 1.         0.03481447 0.03546821]
 [0.0304889  0.03100979 0.04751623 ... 0.03481447 1.         0.03098945]
 [0.0072518  0.01521198 0.01372603 ... 0.03546821 0.03098945 1.        ]]


In [ ]:
similarity.shape

(4803, 4803)

Getting the movie name from the user

In [ ]:
movie_name = input('Enter your favourite  movie name : ')

Enter your favourite  movie name : hulk


In [ ]:
#Creating a list of all the movies in the dataset
list_of_all_titles = df['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [ ]:
#finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Hulk']


In [ ]:
close_match = find_close_match[0]
print(close_match)

Hulk


In [ ]:
#finding the index of the movie with title
index_of_the_movie = df[df.title==close_match]['index'].values[0]
print(index_of_the_movie)

165


In [ ]:
#getting a list of similar movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.033381632544271976), (1, 0.012321827461610937), (2, 0.03543106322295893), (3, 0.015310022145780824), (4, 0.028682718172830995), (5, 0.04524478832866358), (6, 0.014264671578844958), (7, 0.03227744238547808), (8, 0.01911770390181533), (9, 0.030710551712514318), (10, 0.0299815199275927), (11, 0.017442862189610737), (12, 0.05155408998366886), (13, 0.014651283131049313), (14, 0.059296755155105296), (15, 0.011749938623740824), (16, 0.035164782871378565), (17, 0.04655582439030395), (18, 0.0773129274811062), (19, 0.011331809083408605), (20, 0.043446136946892366), (21, 0.01971172849691689), (22, 0.00413723673019333), (23, 0.051182017368182585), (24, 0.014334130592839018), (25, 0.011170369666975787), (26, 0.043355796299665114), (27, 0.03389925352224845), (28, 0.0596470570622036), (29, 0.022923731950651047), (30, 0.03954574983420622), (31, 0.05694074091152298), (32, 0.016648511863647436), (33, 0.022053534700257546), (34, 0.045624554183908574), (35, 0.06776206130650247), (36, 0.027177553493

In [ ]:
len(similarity_score)

4803

In [ ]:
#Sorting the movie based on their similarity score
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse=True)
print(sorted_similar_movies)

[(165, 0.9999999999999998), (97, 0.13222137654460725), (174, 0.1279844280548786), (4401, 0.11961618342326388), (1920, 0.10967039766820483), (1826, 0.10330018976756988), (421, 0.09300914063568397), (1238, 0.09297960569869843), (2717, 0.09236771213035597), (189, 0.09137453751561932), (3311, 0.09062218959437675), (2967, 0.09059315991913795), (1287, 0.0874223385358037), (4240, 0.08738677694407265), (1111, 0.0855017126791487), (593, 0.08457881743279214), (1762, 0.08420820846679508), (1913, 0.08370069161293513), (168, 0.08227922378826205), (3580, 0.0821825778642365), (3821, 0.08193031353245032), (2157, 0.08116241126045107), (953, 0.08094389981212656), (1200, 0.08005890458035461), (43, 0.07858046274528714), (1241, 0.07842094716849346), (366, 0.0783070669758848), (1708, 0.07807970844635057), (1586, 0.07807756984995722), (1526, 0.07764011585864752), (18, 0.0773129274811062), (704, 0.07683072402227101), (4234, 0.0757314377000613), (589, 0.07511050268942436), (2452, 0.0748302654134392), (1617, 0.

In [ ]:
#Print the name of similar movies based on the index
print('Movies suggested for you : \n')
i=1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if (i<30):
    print(i, '.', title_from_index)
    i+=1

Movies suggested for you : 

1 . Hulk
2 . Shin Godzilla
3 . The Incredible Hulk
4 . The Helix... Loaded
5 . Warrior
6 . Cloverfield
7 . Zodiac
8 . The Island of Dr. Moreau
9 . Brokeback Mountain
10 . Noah
11 . Thank You for Smoking
12 . E.T. the Extra-Terrestrial
13 . A Monster in Paris
14 . My Name Is Bruce
15 . Victor Frankenstein
16 . The Dilemma
17 . Dark City
18 . Draft Day
19 . Final Fantasy: The Spirits Within
20 . Doug's 1st Movie
21 . The Good Heart
22 . Daybreakers
23 . Gremlins 2: The New Batch
24 . The Living Daylights
25 . Terminator Salvation
26 . Sweet November
27 . Hollow Man
28 . The Wedding Planner
29 . A View to a Kill


Movies Recommendation System

In [ ]:
movie_name = input(' Enter your favourite movie name : ')
list_of_all_titles = df['title'].tolist()
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
close_match = find_close_match[0]
index_of_the_movie = df[df.title==close_match]['index'].values[0]
similarity_score = list(enumerate(similarity[index_of_the_movie]))
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse=True)
print('Movies suggested for you : \n')
i=1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if(i<30):
    print(i, '.', title_from_index)
    i+=1

 Enter your favourite movie name : xxx
Movies suggested for you : 

1 . xXx
2 . Face/Off
3 . Mission: Impossible - Ghost Protocol
4 . Babylon A.D.
5 . The Andromeda Strain
6 . Diamonds Are Forever
7 . Saving Private Ryan
8 . Mission: Impossible II
9 . Land of the Dead
10 . Pandorum
11 . I Spy
12 . Lord of War
13 . G-Force
14 . The Man from U.N.C.L.E.
15 . Gangster Squad
16 . Criminal
17 . Renaissance Man
18 . xXx: State of the Union
19 . Green Zone
20 . Star Trek Into Darkness
21 . Faster
22 . Machete Kills
23 . The Iron Giant
24 . Topaz
25 . Bangkok Dangerous
26 . Behind Enemy Lines
27 . Rambo: First Blood Part II
28 . Furious 7
29 . Pitch Black
